# Fake News detection

## Libraries installation

## Libraries et drive mounting

In [ ]:
import pathlib

from typing import Dict

import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from transformers import RobertaTokenizer, RobertaModel, RobertaConfig

from torch.utils.data import Dataset, DataLoader, random_split
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl

from typing import Tuple, Any

from introduction_ia_terminale.dataset.fake_news_dataset import FakeNewsDataset
from introduction_ia_terminale.models.fake_news_detector import FakeNewsDetector
from introduction_ia_terminale.preprocessing.fake_news_preprocessor import FakeNewsPreprocessor

## Load data

In [ ]:
preprocessor = FakeNewsPreprocessor(root_path=pathlib.PurePath('/content/drive/MyDrive/datasets/fake_news/'))
df = preprocessor.dataframe

## Répartition des sujets fake news

In [ ]:
df[df['is_fake'] == 1]['subject'].hist();

Ensemble des news maintenant

In [ ]:
df['subject'].hist();

## Répartition de la longueur des articles

In [ ]:
df['len_content'] = df['content'].apply(lambda content: len(content))

fig = plt.figure(figsize=(25, 10))
sns.kdeplot(data=df, x='len_content', log_scale=True, hue='is_fake');

La répartition en longueur de texte est également bien répartie ! On va pouvoir réfléchir à la stratégie de tokénization pour pouvoir classifier les articles, selon le contenu et le type d'article !

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Plusieurs possibilités sont envisageables, la première serait de récupérer le code de l'encodeur BERT et d'utiliser uniquement la sortie de la dernière couche d'encodeur et moyenner la sortie de l'encodeur pour tous les chunks de texte. Sinon, on ne considère que la première partie du contenu (les 512 premiers tokens), puis that's it !

On peut commencer par la deuxième solution qui est plus simple à implémenter.

Pour construire notre Dataset et notre DataLoader, nous avons besoin de créer les tenseurs puis de les affecter directement au dataset, pré-tokéni

In [ ]:
contents = df['content'].tolist()
targets = df['is_fake'].tolist()

tokenized_contents = tokenizer(contents, truncation=True, padding=True, return_tensors='pt')

In [ ]:
tokenized_contents.keys()

In [ ]:
dataset = FakeNewsDataset(tokenized_contents['input_ids'], tokenized_contents['attention_mask'], torch.FloatTensor(targets))
dataset_size = len(dataset)

train_ratio = 0.9
valid_ratio = 0.08
train_size = int(train_ratio * dataset_size)
valid_size = int(valid_ratio * dataset_size)
test_size = dataset_size - train_size - valid_size

train_set, valid_set, test_set = random_split(dataset, [train_size, valid_size, test_size])

train_loader = DataLoader(train_set, batch_size=128, num_workers=4)
valid_loader = DataLoader(valid_set, batch_size=128, num_workers=4)
test_loader = DataLoader(test_set, batch_size=64, num_workers=4)

In [ ]:
model = FakeNewsDetector()
trainer = pl.Trainer(accelerator='gpu', max_epochs=3)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

In [ ]:
trainer.predict(model, test_loader)